In [ ]:
 pip install --upgrade transformers


In [ ]:
!pip install datasets -U


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
# Incorrect Line - Remove this:
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Correct Line:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW  # Corrected Import


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW  # Corrected Import

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/IndicSentiment", name="translation-hi")

The repository for ai4bharat/IndicSentiment contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/IndicSentiment.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


hi.json:   0%|          | 0.00/178k [00:00<?, ?B/s]

hi.json:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['GENERIC CATEGORIES', 'CATEGORY', 'SUB-CATEGORY', 'PRODUCT', 'BRAND', 'ASPECTS', 'ASPECT COMBO', 'ENGLISH REVIEW', 'LABEL', 'INDIC REVIEW'],
        num_rows: 156
    })
    test: Dataset({
        features: ['GENERIC CATEGORIES', 'CATEGORY', 'SUB-CATEGORY', 'PRODUCT', 'BRAND', 'ASPECTS', 'ASPECT COMBO', 'ENGLISH REVIEW', 'LABEL', 'INDIC REVIEW'],
        num_rows: 1000
    })
})

In [ ]:
# Use test for training, validation for evaluation
train_texts = dataset["test"]["INDIC REVIEW"]
train_labels = dataset["test"]["LABEL"]
val_texts = dataset["validation"]["INDIC REVIEW"]
val_labels = dataset["validation"]["LABEL"]


In [ ]:
train_texts[1]

'होस्ट अमेजिंग थे और पूरी प्रॉपर्टी से चेरी और सेब के पेड़ों से घिरे पहाड़ के दृश्य के साथ वातावरण आश्चर्यजनक रूप से सुंदर है। पूरी जगह फ्री वाई-फाई भी उपलब्ध है और रेट्स भी सस्ते हैं।'

In [ ]:
# Preprocess and tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        super(SentimentDataset, self).__init__()
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt") #return dict as key and value pair
        # Convert labels to numerical representation if they are strings
        self.labels = torch.tensor([1 if label == 'positive' else 0 for label in labels],dtype=torch.long) # Assuming 'positive' and other labels are strings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)




In [ ]:
train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training loop
model.train()
for epoch in range(3):
    total_loss = 0
    for batch in train_loader:
        # input_ids = batch['input_ids'].to(device)
        # attention_mask = batch['attention_mask'].to(device)
        # labels = batch['labels'].to(device)
        # outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} - Training Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
from sklearn.metrics import accuracy_score
# Evaluation
model.eval()
all_preds = []
all_labels = []
total_loss=0
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {accuracy:.4f},Validation Loss:{total_loss/len(val_loader)}")


Validation Accuracy: 1.0000,Validation Loss:0.0011250059469603002


In [ ]:
from transformers Trainer,TrainingArguments

In [ ]:
training_args = TrainingArguments(
         num_train_epochs=3,
         per_device_train_batch_size=16,
         per_device_eval_batch_size=16,
         warmup_steps=500,
         weight_decay=0.01,
         learning_rate=2e-5,
         eval_strategy='epoch',
         report_to=None
     )

In [ ]:
import numpy as np
def custom_metric(outputs):
  logits, labels = outputs
  preds = np.argmax(logits, axis=-1)
  accuracy = accuracy_score(labels, preds)
  return {"accuracy": accuracy}

In [ ]:
!pip install evaluate

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
import os
os.environ['WANDB_DISABLED'] = True
trainer.train()